In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random
import os 
import sys
import warnings
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import plotly.express as px
import nsepython as nse
from datetime import datetime


## Data Acquisition Component

In [23]:
start_date = "01-May-2014"
end_date = "10-Sep-2023"

nifty50 = "NIFTY 50"
niftyBank = "NIFTY BANK"
niftyIT = "NIFTY IT"

nifty_50 = nse.index_total_returns(nifty50,start_date,end_date)
# print("Nifty 50 returns: ",nifty_50)

nifty_bank = nse.index_total_returns(niftyBank,start_date,end_date)
# print("Nifty Bank returns: ",nifty_bank)

nifty_it = nse.index_total_returns(niftyIT,start_date,end_date)
# print("Nifty IT returns: ",nifty_it)

In [24]:
# convert all data to dataframes
nifty_50 = pd.DataFrame(nifty_50)
nifty_bank = pd.DataFrame(nifty_bank)
nifty_it = pd.DataFrame(nifty_it)

In [25]:
nifty_50

,Index Name,Date,TotalReturnsIndex
0,NIFTY 50,08 Sep 2023,29074.22
1,NIFTY 50,07 Sep 2023,28937.96
2,NIFTY 50,06 Sep 2023,28767.78
3,NIFTY 50,05 Sep 2023,28714.78
4,NIFTY 50,04 Sep 2023,28647.16
...,...,...,...
2311,Nifty 50,08 May 2014,8682.37
2312,Nifty 50,07 May 2014,8672.88
2313,Nifty 50,06 May 2014,8754.69
2314,Nifty 50,05 May 2014,8733.92


In [26]:
nifty_bank

,Index Name,Date,TotalReturnsIndex
0,NIFTY BANK,08 Sep 2023,61924.86
1,NIFTY BANK,07 Sep 2023,61543.56
2,NIFTY BANK,06 Sep 2023,60900.04
3,NIFTY BANK,05 Sep 2023,61068.84
4,NIFTY BANK,04 Sep 2023,61132.07
...,...,...,...
2311,Nifty Bank,08 May 2014,16827.53
2312,Nifty Bank,07 May 2014,16699.52
2313,Nifty Bank,06 May 2014,16735.48
2314,Nifty Bank,05 May 2014,16625.25


In [27]:
nifty_it

,Index Name,Date,TotalReturnsIndex
0,NIFTY IT,08 Sep 2023,43920.47
1,NIFTY IT,07 Sep 2023,43964.79
2,NIFTY IT,06 Sep 2023,43780.45
3,NIFTY IT,05 Sep 2023,43852.28
4,NIFTY IT,04 Sep 2023,43579.42
...,...,...,...
2311,Nifty IT,08 May 2014,10082.73
2312,Nifty IT,07 May 2014,10063.57
2313,Nifty IT,06 May 2014,10325.79
2314,Nifty IT,05 May 2014,10375.15


In [28]:
# for safety purposes, we will save the dataframes to csv files

nifty_50.to_csv("nifty_50.csv")
nifty_bank.to_csv("nifty_bank.csv")
nifty_it.to_csv("nifty_it.csv")

## Data Cleaning and Preparation Component

In [29]:
# Calculate relative percentage change in index return value from start date
# formula : 100 * (return at date of interest - return at start date) / return at start date

# convert the TotalIndexReturn column data to number type first in all dataframes
nifty_50['TotalReturnsIndex'] = pd.to_numeric(nifty_50['TotalReturnsIndex'])
nifty_bank['TotalReturnsIndex'] = pd.to_numeric(nifty_bank['TotalReturnsIndex'])
nifty_it['TotalReturnsIndex'] = pd.to_numeric(nifty_it['TotalReturnsIndex'])

# calculate relative percentage change in index return value from start date
# start value is not [0] but [last index] because the data is sorted in descending order

nifty_50_start = nifty_50['TotalReturnsIndex'][len(nifty_50['TotalReturnsIndex'])-1]
nifty_bank_start = nifty_bank['TotalReturnsIndex'][len(nifty_bank['TotalReturnsIndex'])-1]
nifty_it_start = nifty_it['TotalReturnsIndex'][len(nifty_it['TotalReturnsIndex'])-1]

nifty_50['RelativePercentageChange'] = 100 * (nifty_50['TotalReturnsIndex'] - nifty_50_start) / nifty_50_start  

nifty_bank['RelativePercentageChange'] = 100 * (nifty_bank['TotalReturnsIndex'] - nifty_bank_start) / nifty_bank_start

nifty_it['RelativePercentageChange'] = 100 * (nifty_it['TotalReturnsIndex'] - nifty_it_start) / nifty_it_start


In [30]:
nifty_50

,Index Name,Date,TotalReturnsIndex,RelativePercentageChange
0,NIFTY 50,08 Sep 2023,29074.22,233.117016
1,NIFTY 50,07 Sep 2023,28937.96,231.555821
2,NIFTY 50,06 Sep 2023,28767.78,229.605989
3,NIFTY 50,05 Sep 2023,28714.78,228.998743
4,NIFTY 50,04 Sep 2023,28647.16,228.223989
...,...,...,...,...
2311,Nifty 50,08 May 2014,8682.37,-0.522002
2312,Nifty 50,07 May 2014,8672.88,-0.630734
2313,Nifty 50,06 May 2014,8754.69,0.306602
2314,Nifty 50,05 May 2014,8733.92,0.068630


In [31]:
nifty_bank

,Index Name,Date,TotalReturnsIndex,RelativePercentageChange
0,NIFTY BANK,08 Sep 2023,61924.86,273.596318
1,NIFTY BANK,07 Sep 2023,61543.56,271.295913
2,NIFTY BANK,06 Sep 2023,60900.04,267.413519
3,NIFTY BANK,05 Sep 2023,61068.84,268.431899
4,NIFTY BANK,04 Sep 2023,61132.07,268.813370
...,...,...,...,...
2311,Nifty Bank,08 May 2014,16827.53,1.521477
2312,Nifty Bank,07 May 2014,16699.52,0.749185
2313,Nifty Bank,06 May 2014,16735.48,0.966134
2314,Nifty Bank,05 May 2014,16625.25,0.301110


In [32]:
nifty_it

,Index Name,Date,TotalReturnsIndex,RelativePercentageChange
0,NIFTY IT,08 Sep 2023,43920.47,318.962925
1,NIFTY IT,07 Sep 2023,43964.79,319.385699
2,NIFTY IT,06 Sep 2023,43780.45,317.627257
3,NIFTY IT,05 Sep 2023,43852.28,318.312452
4,NIFTY IT,04 Sep 2023,43579.42,315.709606
...,...,...,...,...
2311,Nifty IT,08 May 2014,10082.73,-3.819562
2312,Nifty IT,07 May 2014,10063.57,-4.002331
2313,Nifty IT,06 May 2014,10325.79,-1.500982
2314,Nifty IT,05 May 2014,10375.15,-1.030130


In [36]:
#Combine the updated index data to create a single data frame. Rename the RelativePercentageChange and TotalReturnsIndex columns to distinguish the relative percent change data for the three index funds from each other. 

# rename the columns
nifty_50 = nifty_50.rename(columns={'TotalReturnsIndex': 'nifty50_totalReturnsIndex', 'RelativePercentageChange': 'nifty50_relative_percentage_change'})
nifty_bank = nifty_bank.rename(columns={'TotalReturnsIndex': 'niftyBank_totalReturnsIndex', 'RelativePercentageChange': 'niftyBank_relative_percentage_change'})
nifty_it = nifty_it.rename(columns={'TotalReturnsIndex': 'niftyIT_totalReturnsIndex', 'RelativePercentageChange': 'niftyIT_relative_percentage_change'})



# merge the dataframes
df = pd.merge(nifty_50, nifty_bank, how='inner', on='Date')
df = pd.merge(df, nifty_it, how='inner', on='Date')

# drop index Name_x, index Name_y, index Name_z columns
df = df.drop(['Index Name_x', 'Index Name_y', 'Index Name'], axis=1)

df

,Date,nifty50_totalReturnsIndex,nifty50_relative_percentage_change,niftyBank_totalReturnsIndex,niftyBank_relative_percentage_change,niftyIT_totalReturnsIndex,niftyIT_relative_percentage_change
0,08 Sep 2023,29074.22,233.117016,61924.86,273.596318,43920.47,318.962925
1,07 Sep 2023,28937.96,231.555821,61543.56,271.295913,43964.79,319.385699
2,06 Sep 2023,28767.78,229.605989,60900.04,267.413519,43780.45,317.627257
3,05 Sep 2023,28714.78,228.998743,61068.84,268.431899,43852.28,318.312452
4,04 Sep 2023,28647.16,228.223989,61132.07,268.813370,43579.42,315.709606
...,...,...,...,...,...,...,...
2311,08 May 2014,8682.37,-0.522002,16827.53,1.521477,10082.73,-3.819562
2312,07 May 2014,8672.88,-0.630734,16699.52,0.749185,10063.57,-4.002331
2313,06 May 2014,8754.69,0.306602,16735.48,0.966134,10325.79,-1.500982
2314,05 May 2014,8733.92,0.068630,16625.25,0.301110,10375.15,-1.030130


## Data Visualization Component

In [37]:
# Use plotly express to create line plots of the relative percentage change in the three index funds over time, on a single chart.

# plotly express line plot
fig = px.line(df, x="Date", y=["nifty50_relative_percentage_change", "niftyBank_relative_percentage_change", "niftyIT_relative_percentage_change"], title='Relative Percentage Change in Index Funds over Time')
fig.show()